In [2]:
from keras import layers
from sklearn.model_selection import train_test_split

2024-10-23 07:29:46.809368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 07:29:46.982435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 07:29:47.030535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 07:29:47.353641: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 07:29:50.877923: W tensorflow/compiler/tf2

In [3]:
import keras
from keras import ops

In [ ]:
import numpy as np
import pandas as pd
import unicodedata, re
import tensorrt as trt
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [5]:
def preprocess(tx):
    txt = unicodedata.normalize('NFD', str(tx))
    txt = ''.join([char for char in txt if not unicodedata.combining(char)])
    txt = txt.lower()
    txt = re.sub(r"²+", "", txt)
    txt = re.sub(r"/?\s*ref\s*\.?\s*[a-zA-Z0-9]+", "", txt)
    txt = re.sub(r'(\d)\s*,\s*(\d)', r'\1.\2', txt)
    txt = re.sub(r'(?<=\d)(?=\D)|(?<=\D)(?=\d)', ' ', txt)
    txt = re.sub(r'(\d+)(x)(\d+)', r'\1 \2 \3', txt)

    txt = re.sub(r'(x)(mm|cm)', r' \1 \2', txt)
    txt = re.sub(r'(mm|cm)(x)', r' \1 \2', txt)

    txt = re.sub(r'[^\w\s\./]', '', txt)

    txt = re.findall(r'\d+|\w+|[./]', txt)
    txt = ' '.join(txt)

    return txt

In [6]:
root_path = "../../data/"
df_ = pd.read_csv(root_path+"df_nondim.csv")
df = pd.read_csv(root_path+"df.csv")
df = pd.concat([df, df_], ignore_index=True)
df = df[
    df.category.str.contains("PISOS >|PORCELANATOS >|REVESTIMENTOS >", case=False)
    & ~df.category.str.contains("ACESSÓRIOS PARA PISOS", case=False)]

df_leroy = pd.read_csv(root_path+"df_piso_leroy.csv")

df = pd.concat([df, df_leroy], ignore_index=True)
df.reset_index(drop=True, inplace=True)
df = df[["id", "name", "price"]]
#df["name"] = df["name"].apply(preprocess)
df.drop_duplicates(inplace=True)
display(df.head())
df.shape

,id,name,price
0,999348.0,Porcelanato Calacatta Gold 100x100 Acetinado R...,117.9
1,999707.0,Piso Esmaltado Parquet Brilhante 46x46 Tipo A ...,27.9
2,999100.0,Porcelanato Georgia Bege Cetim Acetinado Retif...,79.9
3,999467.0,Porcelanato Esmaltado HD Fior Di Bosco Acetina...,99.9
4,999090.0,Porcelanato Travertino Bege Cetim Acetinado Re...,79.9


(2541, 3)

In [11]:
df.name.to_list()[:10]

['Porcelanato Calacatta Gold 100x100 Acetinado Retificado Tipo A ELIZABETH / REF. 8055757',
 'Piso Esmaltado Parquet Brilhante 46x46 Tipo A INCENOR / REF. PSI 67040',
 'Porcelanato Georgia Bege Cetim Acetinado Retificado 80x80 Tipo A INCESA / REF. CO0865E1',
 'Porcelanato Esmaltado HD Fior Di Bosco Acetinado 101x101 Tipo A ELIZABETH / REF. 8052889',
 'Porcelanato Travertino Bege Cetim Acetinado Retificado 80x80 Tipo A INCESA / REF. CO0027E1',
 'Porcelanato Capim Dourado 100x100 Acetinado Retificado Tipo A ELIZABETH / REF. 8055779',
 'Porcelanato Tenerife Natural Externo Retificado 26x106 Tipo A INCESA / REF. C61845A1',
 'Porcelanato Travertino Bege Externo Retificado 80x80 Tipo A INCESA / REF. CO0027X1',
 'Porcelanato Calacatta Gold 100x100 Polido Retificado Tipo A ELIZABETH / REF. 8055783',
 'Porcelanato Tenerife Natural Cetim Acetinado Retificado 26x106 Tipo A INCESA / REF. C61845N1']

In [ ]:
['Porcelanato Calacatta Gold 100x100 Acetinado Retificado Tipo A ELIZABETH',
 'Piso Esmaltado Parquet Brilhante 46x46 Tipo A INCENOR',
 'Porcelanato Georgia Bege Cetim Acetinado Retificado 80x80 Tipo A INCESA',
 'Porcelanato Esmaltado HD Fior Di Bosco Acetinado 101x101 Tipo A ELIZABETH',
 'Porcelanato Travertino Bege Cetim Acetinado Retificado 80x80 Tipo A INCESA',
 'Porcelanato Capim Dourado 100x100 Acetinado Retificado Tipo A ELIZABETH',
 'Porcelanato Tenerife Natural Externo Retificado 26x106 Tipo A INCESA',
 'Porcelanato Travertino Bege Externo Retificado 80x80 Tipo A INCESA',
 'Porcelanato Calacatta Gold 100x100 Polido Retificado Tipo A ELIZABETH',
 'Porcelanato Tenerife Natural Cetim Acetinado Retificado 26x106 Tipo A INCESA']

In [6]:
root_path = "../../data/"

df = pd.read_csv(root_path+"df.csv")
df = df[["name", "brand", "price"]]
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
df_clean = df.copy()
df["name"] = df["name"].apply(preprocess)
print(df.shape)
df.head()

(10183, 3)


,name,brand,price
0,telha ecologica classica fit 200 x 75 cm verme...,ONDULINE,85.9
1,pneu 325 aro 8 com 2 lonas leve colson,COLSON,32.9
2,caixa para massa de plastico 20 litros dimax,DIMAX BR,24.9
3,esquadro em aco 14 polegadas x 35 . 5 cm com c...,DIMAX BR,18.9
4,esquadro em aco 12 polegadas x 30 . 4 cm com c...,DIMAX BR,17.9


In [13]:
class PositionEmbeddingFixedWeights(keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim, **kwargs):
        super(PositionEmbeddingFixedWeights, self).__init__(**kwargs)
        word_embedding_matrix = self.position_encoding(vocab_size, output_dim)
        position_embedding_matrix = self.position_encoding(sequence_length, output_dim)
        
        self.word_embedding_layer = layers.Embedding(
            input_dim=vocab_size, output_dim=output_dim,
            weights=[word_embedding_matrix],
            trainable=False
        )
        self.position_embedding_layer = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim,
            weights=[position_embedding_matrix],
            trainable=False
        )

    def position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P


    def call(self, inputs):
        position_indices = tf.range(tf.shape(inputs)[-1])
        embedded_words = self.word_embedding_layer(inputs)
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_words + embedded_indices

In [15]:
batch_size = 32
sequence_len = 11
txt_multi_vectorization = layers.TextVectorization(
    ngrams=None,
    output_mode="int",
    split="whitespace",
    standardize=None,
    output_sequence_length=sequence_len
)

def make_dataset(x):
    X, label = x["name"].tolist(), x["price"].tolist()
    dataset = tf.data.Dataset.from_tensor_slices((X, label))
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda x, y: (txt_multi_vectorization(x), y),
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    return dataset.prefetch(tf.data.AUTOTUNE)

train, test = train_test_split(df, test_size=0.188, random_state=42)
train, val = train_test_split(train, test_size=0.177, random_state=42)

print(f"train: {train.shape}, test: {test.shape}, val: {val.shape}")
#print(f"train: {y_train.shape}, test: {y_test.shape}, val: {y_val.shape}")

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

txt_multi_vectorization.adapt(train["name"].tolist())

train_ds = make_dataset(train)
test_ds  = make_dataset(test)
vals_ds  = make_dataset(val)

train: (1697, 3), test: (478, 3), val: (366, 3)


In [19]:
vocab_size=txt_multi_vectorization.vocabulary_size()

In [16]:
class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, seq_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.seq_length = seq_length
        
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim,
            output_dim=output_dim)
        self.posit_encoding = self.attention_position(
            seq_length,
            output_dim)
        
    def compute_mask(self, *args, **kwargs):
        return self.token_embeddings.compute_mask(*args, **kwargs)

    def call(self, x):
        length = ops.shape(x)[-1]
        x = self.token_embeddings(x)
    
        x *= ops.sqrt(ops.cast(self.output_dim, tf.float32))
        x += self.posit_encoding[tf.newaxis, :length, :]
        return x

    def attention_position(self, length, depth):
        positions = np.arange(length)[:, np.newaxis]
        depth_indices = np.arange(depth)[np.newaxis, :] / depth
        angle_rates = 1 / (1e4 ** (depth_indices / 2))
        angle_rads = positions * angle_rates
    
        pos_encoding = np.zeros_like(angle_rads)
        pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
        pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])
        return tf.cast(pos_encoding, dtype=tf.float32)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim,
            "output_dim": self.output_dim,
            "seq_length": self.seq_length,
        })
        return config

In [17]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.1, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        if embed_dim % num_heads != 0:
            raise ValueError(f"embed_dim ({embed_dim}) deve ser divisível por num_heads ({num_heads})")

    def build(self, input_shape):
        # Multi-head attention
        self.attention = layers.MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.embed_dim,
            dropout=self.dropout_rate
        )

        # Feed-forward network
        self.dense_proj = keras.Sequential([
            layers.Dense(self.dense_dim, activation="relu"),
            layers.Dropout(self.dropout_rate),
            layers.Dense(self.embed_dim)
        ])

        # Layer normalizations
        self.layernorm_1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_2 = layers.LayerNormalization(epsilon=1e-6)

        # Dropout layers
        self.dropout1 = layers.Dropout(self.dropout_rate)
        self.dropout2 = layers.Dropout(self.dropout_rate)

        super(TransformerEncoder, self).build(input_shape)

    def call(self, inputs, training=None, mask=None):
        # Aplicar layer norm antes da attention (pre-norm)
        normalized = self.layernorm_1(inputs)

        # Multi-head attention
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=normalized,
            value=normalized,
            key=normalized,
            attention_mask=padding_mask,
            training=training
        )

        # Residual connection e dropout após attention
        attention_output = self.dropout1(attention_output, training=training)
        out1 = inputs + attention_output

        # Feed-forward network com pre-norm
        normalized_2 = self.layernorm_2(out1)
        ff_output = self.dense_proj(normalized_2)

        # Residual connection e dropout após feed-forward
        ff_output = self.dropout2(ff_output, training=training)
        return out1 + ff_output

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

In [20]:
embed_dim, dense_dim, num_heads = 256, 512, 8
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_len, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.1)(x)
#x = layers.BatchNormalization()(x)

outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(
    optimizer=optimizer,
    loss="mae",
    metrics=["r2_score", "mean_absolute_percentage_error"]
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding            │ (None, None, 256)      │       352,256 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │     2,367,488 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,720,001 (10.38 MB)

 Trainable params: 2,720,001 (10.38 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint("regressor_model.keras",
                                    save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss',
                                  patience=6,
                                  restore_best_weights=True)
]

histories = model.fit(
    train_ds,
    epochs=30,
    callbacks=callbacks,
    validation_data=vals_ds,
)
model_hp = keras.models.load_model(
    "regressor_model.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"MAE: {model_hp.evaluate(test_ds)}")

Epoch 1/30


I0000 00:00:1729621600.373796    8154 service.cc:146] XLA service 0x790b38007af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729621600.373817    8154 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2024-10-22 15:26:40.457551: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-22 15:26:40.841610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:536] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2024-10-22 15:26:40.841668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:540] Memory usage: 5242880 bytes free, 6020661248 bytes total.
2024-10-22 15:26:40.841704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:550] Possibly insufficient driver version: 560.35.3
2024-10-22 15:26:41.196536: E external/local_xla/xla/stream_executor/cuda/cud

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/laccan/IdeaProjects/Orion/ovenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/laccan/IdeaProjects/Orion/ovenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/laccan/IdeaProjects/Orion/ovenv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/laccan/IdeaProjects/Orion/ovenv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/laccan/IdeaProjects/Orio